# Исследование надежности заемщиков.

**Заказчик** — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель проекта** - ответить на 4 вопроса:
- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

**Ход проекта**\
В ходе проекта проверим исходные данные, при необходимости проведем их обработку, затем с помощью сводных таблиц ответим на поставленные вопросы и сделаем выводы.
Основные этапы проекта:
- Обзор данных,
- Предобработка данных,
- Ответы на вопросы,
- Выводы.

## Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data.csv')
#выведем на экран первые 10 строк дата-фрейма, чтобы оценить его структуру
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
#общая информация о данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в файле 21525 записей. По сводной информации видим, что пропущенные данные есть в столбцах `days_employed` и `total_income`, причем их количество одинаково, возможно это не случайно. Кроме того по первым строкам дата-фрейма видим, что столбец `days_employed` содержит отрицательные значения, чего по смыслу быть не может, т.к. это трудовой стаж. Также этот столбец должен содержать данные типа `int64`, а не `float64`, т.к. нелогично учитывать стаж в долях дня.


## Шаг 2. Предобработка данных

### Шаг 2.1 Исправления отрицательных значений.

Заполнять пропущенные значения в количественных данных логично средним или медианным значением, чтобы они могли участвовать в дальнейших расчетах. Но для того, чтобы посчитать это значение, необходимо исправить отрицательные значения в столбце `days_employed`. Сначала посчитаем их количество.

In [4]:
data[data['days_employed'] < 0]['days_employed'].count()

15906

Отрицательных значений больше половины. Значит, их нельзя просто отбросить. Судя по всему, значения правильные, лишь содержат знак "минус". Вероятно, ошибка возникла при ручном вводе информации.
Заменим их на абсолютные значения.

In [5]:
data['days_employed'] = abs(data['days_employed'])
#проверка
data[data['days_employed'] < 0]['days_employed'].count()

0

Найдем статистические показатели по столбцу `days_employed`

In [6]:
days_min = data['days_employed'].min()
days_max = data['days_employed'].max()
days_mean = data['days_employed'].mean()
days_median = data['days_employed'].median()
print(f"Минимальное значение {days_min/365} лет")
print(f"Максимальное значение {days_max/365} лет")
print(f"Среднее значение {days_mean/365} лет")
print(f"Медианное значение {days_median/365} лет")

Минимальное значение 0.06614146093282515 лет
Максимальное значение 1100.6997273296713 лет
Среднее значение 183.32802440225305 лет
Медианное значение 6.0115631969279315 лет


Минимальное значение стажа очень мало, но теоретически возможно. 
А вот максимальное значение 1100 лет нереально большое. Из-за этого среднее значение тоже принимает нереальное значение. 
Можно было бы заменить слишком большие значения медианными, но данный столбец в дальнейшем анализе не участвует, поэтому можем оставить данные, как есть.

### Шаг 2.2 Заполнение пропусков

Проверим общее количество пропущенных значений

In [7]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенных значений около 10%. Проверим, есть ли связь между пропущенными данными в столбцах `days_employed` и `total_income`. Для начала выведем на экран несколько строк с пропущенным значением `days_employed`

In [8]:
data[data['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Похоже, что пропуски в одних и тех же строках. Проверим это.

In [9]:
data_null = data[data['total_income'].isna()]
data_null['days_employed'].unique()

array([nan])

Действительно, получилось, что в дата-фрейме с отсутствующими данными в столбце стажа все данные в столбце дохода так же отсутствуют. Значит, эти пропуски не случайны. Но никаких взаимосвязей с другими столбцами не видно - в остальных столбцах встречаются все варианты значений, это видно даже по 10 первым строкам. Вероятно, кто-то не захотел предоставлять о себе такую информацию. Хотя при подаче заявки на выдачу кредита эта информация является обязательной. Может быть, пропуски возникли из-за технического сбоя.

Заполним пропущенные значения в столбце `days_employed` посчитанным выше медианным значением.

In [10]:
data['days_employed']=data['days_employed'].fillna(days_median)
#проверим
data['days_employed'].isna().sum()

0

Пропуски в столбце `total_income` следует заполнять медианным значением, но не по всему дата-фрейму, а по категориям занятости, т.к. доход пенсионера в общем случае отличается от дохода предпринимателя. Найдем медианные значения дохода для каждой категории занятости.

In [11]:
income_group = data.groupby('income_type')['total_income'].median()
income_group

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Так и есть - медианы по категориям различаются.

In [12]:
#список уникальны значений в столбце категория занятости:
i_type = data['income_type'].unique()
i_type

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Для каждой категории занятости заполним пропуски в данных о доходе медианным значением. Для этого воспользуемся циклом.

In [13]:
for i_type in i_type:
    data.loc[data['income_type'] == i_type, 'total_income'] = data['total_income'].fillna(data.loc[data['income_type'] == i_type, 'total_income'].median())

Снова проверим пропуски в данных.

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

От пропусков в данных избавились.

### Шаг 2.3 Проверка данных на аномалии и исправления.

Проанализируем данные в каждом столбце, чтобы проверить корректность данных.
Начнем со столбца `children`

In [15]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Явно некорректное отрицательное количество детей. Ошибка возможна из-за ручного ввода данных. Заменим значение -1 на 1.
Также настораживает значение 20 детей. Такое, конечно, бывает, но не в 76 случаях из 21 тыс. Здесь возможна опечатка в виде лишнего нуля. Заменим значение 20 на 2.  

In [16]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
#проверим
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Замена выполнена успешно.

Столбец `days_employed` уже обработан ранее.

Следующий столбец `dob_years`

In [17]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Здесь некорректным значением является 0, т.к. речь о возрасте заемщиков. Наиболее правильным будет заменить 0 на медианное значение возраста. Для количественных переменных это наилучшее решение, т.к. медиана показывает среднее по частоте значение.

In [18]:
data.loc[data['dob_years'] == 0, ['dob_years']] = data['dob_years'].median()
#проверим
data['dob_years'].unique()

array([42., 36., 33., 32., 53., 27., 43., 50., 35., 41., 40., 65., 54.,
       56., 26., 48., 24., 21., 57., 67., 28., 63., 62., 47., 34., 68.,
       25., 31., 30., 20., 49., 37., 45., 61., 64., 44., 52., 46., 23.,
       38., 39., 51., 59., 29., 60., 55., 58., 71., 22., 73., 66., 69.,
       19., 72., 70., 74., 75.])

Ноль исчез из значений. Зато при подстановке медианного значения изменился тип данных на вещественный. Но это не проблема, в следующем шаге мы поменяем его обратно на целочисленный.

Следующий столбец для проверки `education`

In [19]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Здесь видим, что одни и те же позиции записаны с применением строчных и заглавных букв. Приведем все значения к нижнему регистру.

In [20]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Оказалось, уникальных значений всего 5.

Следующий столбец для проверки `education_id`

In [21]:
data['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

Здесь все в порядке.

Проверим следующий столбец `family_status`.

In [22]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

С этим столбцом тоже все в порядке.

Проверим `family_status_id`.

In [23]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

Здесь тоже все в порядке. Следующий `gender`

In [24]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Будем считать, что один раз встречающееся `XNA` - это "неопределенный пол", такое тоже бывает.

Значения столбца `income_type` мы уже получали, когда считали медианный доход по категориям. Там все в норме. Столбец `total_income` уже обработан. 

Следующий проверим `debt`.

In [25]:
data['debt'].unique()

array([0, 1], dtype=int64)

Здесь все, как и должно быть.

Последний столбец `purpose`.

In [26]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Здесь много однотипных значений. Обработаем их позже.

### Шаг 2.4. Изменение типов данных.

В самом начале обзора данных упоминалось, что нужно изменить тип данных в столбце `days_employed` с `float64` на `int64`. Сделаем это. Также изменим на целочисленный тип данные в столбце `dob_years`, где тип данных изменился из-за подстановки медианы. Также для удобства восприятия изменим на целочисленный тип данных в столбце `total_income`.

In [27]:
data[['days_employed', 'dob_years', 'total_income']] = data[['days_employed', 'dob_years', 'total_income']].astype('int')
#проверим
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int32 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(3), int64(4), object(5)
memory usage: 1.7+ MB


Теперь все корректно.

### Шаг 2.5. Удаление дубликатов.

Проверим датафрейм на наличие явных дубликатов.

In [28]:
data.duplicated().sum()

72

В данных 72 дублирующих строки. Удалим дублирующиеся строки с обновлением индексов.

In [29]:
data = data.drop_duplicates().reset_index(drop=True)
#проверим
data.duplicated().sum()

0

Неявные дубликаты (если они были), образовавшиеся за счет разного регистра в столбце образование, удалились сейчас автоматически, т.к. мы заранее преобразовали регистр.

### Шаг 2.6. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

В процессе анализа данных удобно пользоваться индексами, а не строковыми значениями. Вынесем строковые значения в столбцах `education` и `family_status` в отдельные словари, а в основном дата-фрейме будем пользоваться идентификаторами.

In [30]:
education_dict = data.loc[:, ['education', 'education_id']]
family_dict = data.loc[:, ['family_status', 'family_status_id']]
display(education_dict.head())
display(family_dict.head())

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1


,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1


Удалим дубликаты в словарях.

In [31]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
display(education_dict.head())
display(family_dict.head())

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Теперь у нас готовы словари. Данные из основного дата-фрейма сохраним отдельно только с идентификаторами образования и семейного положения.

In [32]:
data_log = data.drop(['education', 'family_status'], axis=1)
display(data_log.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.7. Категоризация дохода.

  Чтобы было удобно анализировать доход, разобьем его на категории. Для этого напишем функцию, которая на вход получит значение ежемесячного дохода, а на выходе вернет категорию клиента по следующему правилу:\
0–30000 — 'E';\
30001–50000 — 'D';\
50001–200000 — 'C';\
200001–1000000 — 'B';\
1000001 и выше — 'A'.  

In [33]:
def income_cat(income):
    if income <= 30000:
        return 'E'
    if income <=50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

#проверим работу функции
print(f'Клиент с доходом 29000 относится к категории {income_cat(29000)}')
print(f'Клиент с доходом 256000 относится к категории {income_cat(256000)}')
print(f'Клиент с доходом 132560 относится к категории {income_cat(132560)}')

Клиент с доходом 29000 относится к категории E
Клиент с доходом 256000 относится к категории B
Клиент с доходом 132560 относится к категории C


Теперь применим функцию к столбцу `total_income` и создадим новый столбец `total_income_category`.

In [34]:
data_log['total_income_category'] = data_log['total_income'].apply(income_cat)
data_log.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.8. Категоризация целей кредита.

Как отмечалось выше при анализе уникальных значений столбца `purpose`, в нем много близких по смыслу значений, которые следует объединить в категории.
Выведем их еще раз на экран и попробуем проанализировать.

In [35]:
data_log['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Все значения сводятся к четырем группам:
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.\
Аналогично предыдущему пункту создадим функцию, которая будет распределять строки на входе по категориям и возвращать название категории.

In [36]:
def purpose_cat(purpose):
    if purpose.find('автомоб') >= 0:
        return 'операции с автомобилем'
    if (purpose.find('недвиж') >= 0) | (purpose.find('жил') >= 0):
        return 'операции с недвижимостью'
    if purpose.find('свад') >= 0:
        return 'проведение свадьбы'
    if purpose.find('образов') >= 0:
        return 'получение образования'
    return 'категория не определена'  # введем эту строку для проверки, все ли мы учли

#проверим работу функции
print(f'"покупка жилья" относится к категории "{purpose_cat("покупка жилья")}"')
print(f'"дополнительное образование" относится к категории "{purpose_cat("дополнительное образование")}"')
print(f'"сделка с подержанным автомобилем" относится к категории "{purpose_cat("сделка с подержанным автомобилем")}"')
print(f'"покупка" относится к категории "{purpose_cat("покупка")}"')

"покупка жилья" относится к категории "операции с недвижимостью"
"дополнительное образование" относится к категории "получение образования"
"сделка с подержанным автомобилем" относится к категории "операции с автомобилем"
"покупка" относится к категории "категория не определена"


Применим теперь эту функцию к столбцу `purpose`

In [37]:
data_log['purpose_category'] = data_log['purpose'].apply(purpose_cat)
#проверим, все ли варианты целей мы учли при категоризации
data_log.loc[data_log['purpose_category'] == 'категория не определена']['purpose_category'].count()

0

В столбце `purpose_category` нет значений `категория не определена`. Т.о. мы все учли, можно приступать к анализу.

In [38]:
data_log.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Вывод по шагу 2

Итак, мы подготовили данные для анализа:
- Заполнили недостающие значения, удалили дубликаты, исправили аномальные значения, изменили типы данных.
- Для упрощения и ускорения процесса анализа сформировали словари идентификаторов по столбцам семейное положение и уровень образования.
- Сгруппировали доход и цели кредита по категориям.

Теперь имеем данные, удобные для формирования сводных таблиц и ответов на поставленные вопросы.

## Шаг 3. Ответы на вопросы

### Вопрос 1:

Определим, есть ли зависимость между количеством детей и возвратом кредита в срок. Для этого создадим сводную таблицу, в строках которой укажем количество детей, в столбцах - показатель возврата кредита, а в ячейках посчитаем количество соответствующих случаев.

In [39]:
data_pivot_children = data_log.pivot_table(index='children', columns='debt', values='purpose', aggfunc='count')
display(data_pivot_children)

debt,0,1
children,,
0,13027.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


`0` означает отсутствие задолженности, `1` - наличие задолженности.\
Пока что видим, что возвращают кредит вовремя примерно в 10 раз чаще, чем имеют задолженности, при любом количестве детей. Также по данной таблице видим, что чем меньше детей, тем цифры больше. Но это говорит лишь о том, что среди заемщиков, как и в обществе в целом, чаще всего встречаются люди без детей, с ростом количества детей число соответствующих семей уменьшается.\
Чтобы ответить на наш вопрос, посчитаем для каждого количества детей долю возврата кредита в срок. В этом нам поможет функция для строки дата-фрейма.

In [40]:
def debt_proportion(row):
    return row[0] /(row[0] + row[1])
#проверим работу функции
row_values = [3, 7]
row_columns = [0, 1]
row = pd.Series(data=row_values, index=row_columns)
debt_proportion(row)

0.3

Теперь применим функцию к сводной таблице. Добавим в ней новый столбец со значением доли вовремя выплаченных кредитов `proportion_0`

In [41]:
data_pivot_children['proportion_0'] = data_pivot_children.apply(debt_proportion, axis=1)
display(data_pivot_children)

debt,0,1,proportion_0
children,,,
0,13027.0,1063.0,0.924556
1,4410.0,445.0,0.908342
2,1926.0,202.0,0.905075
3,303.0,27.0,0.918182
4,37.0,4.0,0.902439
5,9.0,NaN,NaN


In [42]:
data_log.pivot_table(index='children', values='debt')

,debt
children,
0,0.075444
1,0.091658
2,0.094925
3,0.081818
4,0.097561
5,0.000000


#### Вывод 1:

По полученным данным можно сделать вывод о том, что чаще всего (но незначительно) вовремя возвращают кредит заемщики без детей, на втором месте заемщики в тремя детьми, потом все остальные. С пятью детьми вообще все взятые кредиты были возвращены вовремя, но выборка в данном случае слишком мала (9 случаев), чтобы утверждать однозначно.\
Разница в долях незначительна, я бы считала, что зависимости между возвратом кредита в срок и количеством детей практически нет.

### Вопрос 2:

Аналогичным образом определим, есть ли зависимость между семейным положением и возвратом кредита в срок. Создадим сводную таблицу, в строках которой укажем различные варианты семейного положения, в столбцах - показатель возврата кредита, а в ячейках посчитаем количество соответствующих случаев.

In [43]:
data_pivot_family = data_log.pivot_table(index='family_status_id', columns='debt', values='purpose', aggfunc='count')
display(data_pivot_family)

debt,0,1
family_status_id,,
0,11408,931
1,3762,388
2,896,63
3,1110,85
4,2536,274


Воспользуемся написанной ранее функцией `debt_proportion`

In [44]:
data_pivot_family['proportion_0'] = data_pivot_family.apply(debt_proportion, axis=1)
display(data_pivot_family)

debt,0,1,proportion_0
family_status_id,,,
0,11408,931,0.924548
1,3762,388,0.906506
2,896,63,0.934307
3,1110,85,0.928870
4,2536,274,0.902491


По данной таблице сложно что-либо сказать, поскольку в ней присутствуют только идентификаторы семейного положения. Добавим в сюда расшифровки из словаря. И отсортируем сводную таблицу по столбцу `proportion_0` по убыванию.

In [45]:
data_pivot_family = data_pivot_family.merge(family_dict, on='family_status_id', how='left')
data_pivot_family.sort_values('proportion_0', ascending=False)

,family_status_id,0,1,proportion_0,family_status
2,2,896,63,0.934307,вдовец / вдова
3,3,1110,85,0.928870,в разводе
0,0,11408,931,0.924548,женат / замужем
1,1,3762,388,0.906506,гражданский брак
4,4,2536,274,0.902491,Не женат / не замужем


#### Вывод 2:

По полученной таблице легко можно сделать вывод о том, что лучше всех кредиты возвращают вдовцы/вдовы, хуже всех - не женатые/не замужние люди.

### Вопрос 3:

Определим, есть ли зависимость между уровнем дохода и возвратом кредита в срок. Аналогично предыдущим пунктам создадим сводную таблицу и сразу применим к ней функцию подсчета доли вовремя возвращенных кредитов.

In [46]:
data_pivot_income = data_log.pivot_table(index='total_income_category', columns='debt', values='purpose', aggfunc='count')
data_pivot_income['proportion_0'] = data_pivot_income.apply(debt_proportion, axis=1)
data_pivot_income

debt,0,1,proportion_0
total_income_category,,,
A,23,2,0.920000
B,4686,356,0.929393
C,14654,1360,0.915074
D,329,21,0.940000
E,20,2,0.909091


Пока по этой таблице можно лишь заметить, что количество кредитов в зависимости от дохода имеет близкое к нормальному распределение, т.е. люди со средним доходом чаще всего берут кредиты.\
Напомним, как распределялись категории дохода:\
0–30000 — 'E';\
30001–50000 — 'D';\
50001–200000 — 'C';\
200001–1000000 — 'B';\
1000001 и выше — 'A'.\
Создадим для этих категорий словарь, чтобы потом соединить его со сводной таблицей.

In [47]:
cat = [['A', '1000001 и выше'],
       ['B', '200001–1000000'],
       ['C', '50001–200000'],
       ['D', '30001–50000'],
       ['E', '0–30000']]
columns = ['total_income_category', 'total_income']
category = pd.DataFrame(data=cat, columns=columns)
category

,total_income_category,total_income
0,A,1000001 и выше
1,B,200001–1000000
2,C,50001–200000
3,D,30001–50000
4,E,0–30000


Объединим словарь со сводной таблицей и отсортируем по столбцу `proportion_0`

In [48]:
data_pivot_income = data_pivot_income.merge(category, on='total_income_category', how='left')
data_pivot_income.sort_values('proportion_0', ascending=False)

,total_income_category,0,1,proportion_0,total_income
3,D,329,21,0.940000,30001–50000
1,B,4686,356,0.929393,200001–1000000
0,A,23,2,0.920000,1000001 и выше
2,C,14654,1360,0.915074,50001–200000
4,E,20,2,0.909091,0–30000


#### Вывод 3:

По полученным данным сложно сделать однозначный вывод, полученная зависимость нелинейная. Хуже всего возвращают кредиты заемщики с самым маленьким доходом, но выборка по ним самая маленькая и потому не показательна. Наилучший результат тоже в достаточно маленькой выборке. Наиболее показательны цифры для людей со средним доходом, а они практически одинаковые.

### Вопрос 4:

Определим, как разные цели кредита влияют на его возврат в срок. Воспользуемся уже знакомыми методами, аналогичными предыдущим пунктам.

In [49]:
data_pivot_purpose = data_log.pivot_table(index='purpose_category', columns='debt', values='purpose', aggfunc='count')
data_pivot_purpose['proportion_0'] = data_pivot_purpose.apply(debt_proportion, axis=1)
data_pivot_purpose.sort_values('proportion_0', ascending=False)

debt,0,1,proportion_0
purpose_category,,,
операции с недвижимостью,10029,782,0.927666
проведение свадьбы,2137,186,0.919931
получение образования,3643,370,0.907800
операции с автомобилем,3903,403,0.906410


#### Вывод 4:

Чаще всего берут кредит и лучше всего возвращают на операции с недвижимостью. Хуже всего возвращают кредиты на получение образования и операции с автомобилем (практически одинаковый процент возврата).

## Общий вывод:

В ходе проекта мы провели большую часть работы по предобработке данных: заполнили пропущенные значения, удалили дубликаты, заменили аномальные значения, вынесли некоторые признаки в отдельные словари, заменив их при анализе идентификаторами, чтобы сократить требуемые для анализа ресурсы, создали категории, удобные для анализа.\
В целом по итогам анализа можно признать, что на своевременный возврат кредита не влияет ни количество детей, ни семейное положение, ни уровень дохода, ни цель кредита. Во всех случаях результаты различаются в пределах 4% (от 90% до 94%) и находятся в пределах статистической погрешности. Я думаю, при увеличении выборки результаты станут еще более близки друг к другу, в среднем 92%.